# Loading data from preprocessed pickle file

In [1]:
import pickle

# open the pickle file in read-binary mode
with open('..//model-data/preprocessing/train-val-sets/train_val_split.pkl', 'rb') as f:
    data = pickle.load(f)

# storing data
images_train = data['images_train']
images_val = data['images_val']
labels_train = data['labels_train']
labels_val = data['labels_val']

# Creating model

In [ ]:
import tensorflow as tf
import keras
from keras import layers, Sequential

model = keras.Sequential([
    layers.Rescaling(scale=1.0/255, input_shape=(256, 256, 3)),
    
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    
    layers.Flatten(),
    layers.Dense(39, activation='softmax')
])

In [2]:
import tensorflow as tf
import keras
from keras import layers, Sequential

model = keras.Sequential([
    layers.Rescaling(scale=1.0/255, input_shape=(256, 256, 3)),
    
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(39, activation='softmax')
])

2025-01-25 18:06:27.895348: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-25 18:06:27.904934: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-25 18:06:27.919511: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-25 18:06:27.919548: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-25 18:06:27.928237: I tensorflow/core/platform/cpu_feature_gua

In [ ]:
import tensorflow as tf
import keras
from keras import layers, Sequential

model = keras.Sequential([
    layers.Input(shape=(256, 256, 3)),
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.Flatten(),
    layers.Dense(16, activation='softmax'),
    layers.Dense(39, activation='softmax')
])

# Compling the model

In [3]:
# compiling the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fitting the model

In [4]:
# sliding window model training (to handle batching due to tensorflow gpu mem errors)
for w in range(2):
    # calculating a third of the data size
    third_of_data = len(images_train) // 3
    HR_size = third_of_data
    LR_size = 0
    
    for i in range(3):
        #print(LR_size, HR_size)
        images_train_third = images_train[LR_size:HR_size]
        labels_train_third = labels_train[LR_size:HR_size]
    
        model.fit(images_train_third, labels_train_third, epochs=1)
        
        LR_size += third_of_data;
        HR_size += third_of_data

I0000 00:00:1737828392.141666   26462 service.cc:145] XLA service 0x759d74003c90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1737828392.141724   26462 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2025-01-25 18:06:32.190115: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-01-25 18:06:32.341451: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906
I0000 00:00:1737828393.122566   26526 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_765', 544 bytes spill stores, 544 bytes spill loads



  3/440 ━━━━━━━━━━━━━━━━━━━━ 19s 45ms/step - accuracy: 0.1285 - loss: 36.2660

I0000 00:00:1737828396.782591   26462 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


440/440 ━━━━━━━━━━━━━━━━━━━━ 24s 42ms/step - accuracy: 0.4887 - loss: 44.7575
440/440 ━━━━━━━━━━━━━━━━━━━━ 18s 41ms/step - accuracy: 0.7429 - loss: 11.2192
440/440 ━━━━━━━━━━━━━━━━━━━━ 18s 41ms/step - accuracy: 0.7930 - loss: 5.4727
440/440 ━━━━━━━━━━━━━━━━━━━━ 18s 41ms/step - accuracy: 0.8581 - loss: 2.8038
440/440 ━━━━━━━━━━━━━━━━━━━━ 18s 41ms/step - accuracy: 0.8848 - loss: 1.8197
440/440 ━━━━━━━━━━━━━━━━━━━━ 18s 41ms/step - accuracy: 0.8879 - loss: 1.6092


# Validating the model

In [5]:
import numpy as np
from sklearn.metrics import accuracy_score

# converting one hot encoded labels to class indices
real_labels = np.argmax(labels_val, axis=1)

# predict on the validation set
predictions = model.predict(images_val)

# converting predictions from one hot encoded probabiliti|es to class indices
predicted_labels = np.argmax(predictions, axis=1)

# comparing predictions with the real labels
print("Validation Accuracy:", accuracy_score(real_labels, predicted_labels))

331/331 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
Validation Accuracy: 0.8186724741975192


# [TESTING]

In [6]:
import numpy as np

def one_hot_max(arr):
    result = np.zeros_like(arr)
    result[np.argmax(arr)] = 1
    return result

print(one_hot_max(predictions[4]))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [7]:
print(labels_val[4])

[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]
